# Read RateMDs and Yelp

In [ ]:
import os 


import pandas as pd
import os
import numpy as np
pd.set_option('display.max_columns', None)


def get_df_from_folder2rd(subfolder):
    # subfolder = os.path.join(folder, folder2rd)
    filenames = [os.path.join(subfolder,i) for i in os.listdir(subfolder) if 'errorlog' not in i and '.DS_Store' not in i]
    L = []
    for filename in filenames:
        try:
            df = pd.read_pickle(filename)
            L.append(df)
        except:
            print('Error File', filename.split('/')[-1])

    DF = pd.concat(L).reset_index(drop = True)
    return DF
        

In [ ]:

WebsiteNames = ['ratemds',  'yelp']

D = {}
for name in WebsiteNames:
    L = []
    folder = os.path.join('Output', name)
    subfolders = [os.path.join(folder, i) for i in os.listdir(folder) if name in i and '.DS' not in i]# 'Data/DocListDF_GoogleSearch_s0_e500.p'
    subfolders.sort()
    
    for folder2rd in subfolders:
        DF = get_df_from_folder2rd(folder2rd)
        L.append(DF)
        
    FullDF = pd.concat(L).reset_index(drop = True)
    D[name] = FullDF
    
    

In [ ]:
import pandas as pd

NPPES = pd.read_pickle('Data/NPI2InfoMDDO.p')
NPPES.columns

In [ ]:

NPPESNames = NPPES[['NPI', 'LastName', 'FirstName', 'Gender']]
print(NPPESNames.shape)


# Yelp

In [ ]:
yelp_df = D['yelp']
yelp_df.columns

In [ ]:
cols = ['source_npi', 'name']
tmp = yelp_df[cols]
tmp = tmp.rename(columns = {'source_npi':'NPI'})

YelpInfo = pd.merge(tmp, NPPESNames, on = 'NPI')
YelpInfo

In [ ]:
L1 = []
L2 = []
for idx, row in YelpInfo.iterrows():
    try:
        lastname = row['LastName'].lower()
        firstname = row['FirstName'].lower()
        yelpname = row['name'].lower()
    except:
        print(row)
    
    if firstname in yelpname and lastname in yelpname:
        pass
    elif 'md' in yelpname or 'do' in yelpname:
        # print(row.to_dict())
        L1.append(row.to_dict())
    else:
        L2.append(row.to_dict())
        
L1df = pd.DataFrame(L1)
L2df = pd.DataFrame(L2)# .to_csv('YelpInstitution.csv', index = False)
TotalWrong = pd.concat([L1df, L2df])
print(len(TotalWrong)) 
TotalWrong.reset_index(drop = True).to_csv('ToIgnore/ToIgnoreYelp.csv', index = False)

# RateMDs

In [ ]:
ratemd_df = D['ratemds']

ratemd_df_cols = ['source_npi', 'full_name']
tmp = ratemd_df[ratemd_df_cols]
tmp = tmp.rename(columns = {'source_npi':'NPI'})

RateMDInfo = pd.merge(tmp, NPPESNames, on = 'NPI')
RateMDInfo

In [ ]:
L1 = []
L2 = []

for idx, row in RateMDInfo.iterrows():
    try:
        lastname = row['LastName'].lower()
        firstname = row['FirstName'].lower()
        yelpname = row['full_name'].lower()
    except:
        print(row)
    
    if firstname in yelpname and lastname in yelpname:
        pass
    elif 'md' in yelpname or 'do' in yelpname:
        # print(row.to_dict())
        L1.append(row.to_dict())
    else:
        L2.append(row.to_dict())
        
L1df = pd.DataFrame(L1)
L2df = pd.DataFrame(L2)# .to_csv('YelpInstitution.csv', index = False)
TotalWrong = pd.concat([L1df, L2df])
print(len(TotalWrong)) 
TotalWrong.reset_index(drop = True).to_csv('ToIgnore/ToIgnoreRateMDs.csv', index = False)